# Automated Gleason Grading

In [1]:
from lib.load import *
from lib.merged_training import *
from lib.testing import *

In [2]:
train_img_names, val_img_names, test_img_names, thumbnails_dict = load()

In [3]:
unet = mergedTraining(train_img_names, val_img_names, thumbnails_dict)

Make Slide Dict: 100%|██████████| 1486/1486 [00:01<00:00, 894.88it/s]


[INFO] found 256 samples in the merged training set...
[INFO] found 256 samples in the merged validation set...
[INFO] training the network...


  2%|▏         | 1/50 [01:11<58:24, 71.53s/it]

[INFO] EPOCH: 1/50
Train loss: 0.591677, Val loss: 0.4741
Val metric for class 0: 0.000000
Val metric for class 1: 0.000000
Val metric for class 2: 0.669804


  4%|▍         | 2/50 [02:22<56:49, 71.04s/it]

...

[INFO] EPOCH: 48/50
Train loss: 0.230918, Val loss: 0.2182
Val metric for class 0: 0.748429
Val metric for class 1: 0.777416
Val metric for class 2: 0.701840


 98%|█████████▊| 49/50 [1:00:15<01:16, 76.00s/it]

[INFO] EPOCH: 49/50
Train loss: 0.246220, Val loss: 0.2358
Val metric for class 0: 0.738925
Val metric for class 1: 0.771934
Val metric for class 2: 0.649264


100%|██████████| 50/50 [1:01:30<00:00, 73.81s/it]

[INFO] EPOCH: 50/50
Train loss: 0.239552, Val loss: 0.2334
Val metric for class 0: 0.722746
Val metric for class 1: 0.764454
Val metric for class 2: 0.660149
[INFO] total time taken to train the model: 3690.71s


In [ ]:
# Serialize the model to disk
torch.save(unet, MODEL_PATH)

In [4]:
# testingLoss = testing(unet, test_img_names)